<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-Directories" data-toc-modified-id="Imports-and-Directories-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports and Directories</a></span></li><li><span><a href="#Execution" data-toc-modified-id="Execution-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Execution</a></span><ul class="toc-item"><li><span><a href="#Processing-and-saving-the-file" data-toc-modified-id="Processing-and-saving-the-file-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Processing and saving the file</a></span></li></ul></li><li><span><a href="#Get-100k-sample" data-toc-modified-id="Get-100k-sample-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get 100k sample</a></span></li><li><span><a href="#Get-1k-samples" data-toc-modified-id="Get-1k-samples-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Get 1k samples</a></span></li></ul></div>

# Imports and Directories

In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import os

In [2]:
from pathlib import *
#current working directory
current_dir = Path.cwd()
#go up 1 level to the 1st parent directory
parent_dir = current_dir.parents[0]

In [3]:
from IPython.display import Audio, display

def allDone():
    '''this function outputs a short audio when called. 
    Typically this is used to signal a task completion'''
    
    display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))

# Execution

In [4]:
# list used 
temp_list = ['jp morgan', 'jpm', 'jp-m', 'j.p. morgan', 'jpmorgan', 
'etoro', 'e-toro', 'e toroa', 
'visa',
'goldman sachs', 'goldman-sachs', 'goldmansachs',
'unilever',
'deloitte',
'google', 'alphabet',
'wells-fargo', 'wells fargo',
'allianz', 'allianz-inssurance',
'samsung',
'apple',
'commerzbank']

years = ['2013', '2014', '2015', '2016', '2017', '2018', '2019']
#years = ['2013', '2014', '2015']


com_list = [['JP Morgan',['jp morgan', 'jpm', 'jp-m', 'j.p. morgan', 'JPMorgan']],
             ['Etoro',['etoro', 'e-toro', 'e toro']],
             ['Google', ['google', 'alphabet']],
             ['Visa', ['visa']],
             ['Goldman Sachs', ['goldman sachs', 'goldman-sachs', 'goldmansachs']],
             ['Unilever', ['unilever']],
             ['Deloitte', ['deloitte']],
             ['Samsung', ['samsung']],
             ['Wells Fargo', ['wells-fargo', 'wells fargo']],
             ['Allianz', ['allianz', 'allianz-inssurance']],
             ['Apple', ['apple']],
             ['Commerzbank', ['commerzbank']]]
            


company_list = ['JP Morgan', 'Etoro', 'Google', 'Visa', 'Goldman Sachs',
               'Unilever', 'Deloitte', 'Samsung', 'Wells Fargo', 'Allianz',
               'Apple', 'Commerzbank']


# Cut by year
reject2 = ['google.com', 'google play!', 'chinese google']


def current_yr(df, year):
    '''
    df = original df from dask
    year = taken from list
    '''
    df_year = df[df['date/time'].str.contains(year)]
    
    # Source from reference first
    try:
        clean_year = [tweet.lower() for tweet in df_year.tweet if any(tweet for ref in temp_list if str(ref) in tweet.lower())]
    except:
        clean_year = [str(tweet).lower() for tweet in df_year.tweet if any(tweet for ref in temp_list if str(ref) in str(tweet).lower())]
        
    # replace certain terms with empty to remove the companies for misreferenced tweets
    clean_year_01 = clean_year
    for rep in reject2:
        clean_year_01 = [tweet.replace(rep, '') for tweet in clean_year_01]
        
    # Rerun to clean out all the non companies list ref
    clean_year_02 = [tweet for tweet in clean_year_01 if any(tweet for ref in temp_list if str(ref) in tweet)]
    
    # Create the new df
    df_avery = pd.DataFrame(columns = company_list)
    df_avery['tweets'] = clean_year_02

    matrix = [[] for x in range(len(company_list))]

    for i in range(len(com_list)):
        for j in range(len(df_avery)):
            if  any(x in df_avery.tweets[j] for x in com_list[i][1]):
                matrix[i].append(1)

            else:
                matrix[i].append(0)

    df_avery[company_list] = np.transpose(matrix)
    df_avery['year'] = year
    print(year)
    return df_avery




In [5]:
df_year = df[df['date/time'].str.contains('2018')]
df_year.info()

NameError: name 'df' is not defined

In [26]:
df[df['date/time'].str.contains('2016')].head(4)

,date/time,tweet
216903,2016-01-25 23:59:55+00:00,Microsoft Releases Its Deep Learning Toolkit O...
216904,2016-01-25 23:57:52+00:00,Drowning in #opensource these days | Microsoft...
216905,2016-01-25 23:57:51+00:00,"Microsoft releases #CNTK, its #opensource #dee..."
216906,2016-01-25 23:57:45+00:00,Google's free 3 month course on deep learning:...


## Processing and saving the file

In [6]:
# AI Part only

# Data source
data_path = parent_dir /'AI Project'/'Python Env'/'DataFrames'/'AI_datasets'/'globe_ai_13_18/*.csv'

from glob import glob
files = glob(str(data_path))
df_files = [pd.read_csv(file, usecols=['tweet', 'date/time']) for file in files]

In [7]:
df = pd.concat(df_files, ignore_index=True)

In [8]:

df_list = [current_yr(df, year) for year in years]

# Save the file
df_full = pd.concat(df_list, ignore_index=True)

OUTPUT = parent_dir/'AI Project'/'Avery_output'/'AI_sequential_topic'
outname ='AI_topic_company_full.csv'


df_full.to_csv(os.path.join(str(OUTPUT),outname), index = False)


allDone()

2013
2014
2015
2016
2017
2018
2019


In [30]:
str(float(9.00)).lower()

'9.0'

# Get 100k sample

In [9]:
def get_100k(n,df, year):
    '''
    n = number of samples to take
    df= processed df 
    year= year selected
    
    Only collect 100k sample for each year
    '''
    
    df1 = df[df['year'] == year]
    if len(df1) > n:
        df1 = df1.sample(n=n, random_state=11)
        
    else:
        df1 = df1
        print(year)
        print(len(df1))
    
    return df1
    
    

In [37]:
# get 100k samples
df_100_list = [get_100k(n=100000, df=df_full, year=year) for year in years]

df_100 = pd.concat(df_100_list, ignore_index=True)

OUTPUT = parent_dir/'AI Project'/'Avery_output'/'AI_sequential_topic'
outname ='AI_topic_company_100k.csv'


df_100.to_csv(os.path.join(str(OUTPUT),outname), index = False)

In [9]:
OUTPUT = parent_dir/'AI Project'/'Avery_output'/'AI_sequential_topic'
outname ='AI_topic_company_100k.csv'

df_100 = pd.read_csv(os.path.join(str(OUTPUT),outname))

In [10]:
df_100

,JP Morgan,Etoro,Google,Visa,Goldman Sachs,Unilever,Deloitte,Samsung,Wells Fargo,Allianz,Apple,Commerzbank,tweets,year
0,0,0,1,0,0,0,0,0,0,0,0,0,7 common #google analytics mistakes and how to...,2013
1,0,0,1,0,0,0,0,0,0,0,0,0,analyze organic search engine marketing with g...,2013
2,0,0,1,0,0,0,0,0,0,0,0,0,the @visually google analytics report is prett...,2013
3,0,0,1,0,0,0,0,0,0,0,0,0,piwik - outil de mesure libre et open source d...,2013
4,0,0,1,0,0,0,0,0,0,0,0,0,#c-sharp-programming integrate call data into ...,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675174,0,0,1,0,0,0,0,0,0,0,0,0,rt @mldcmu: #facebook joins #amazon and #googl...,2019
675175,0,0,1,0,0,0,0,0,0,0,0,0,#facebook joins #amazon and #google in #ai chi...,2019
675176,0,0,1,0,0,0,0,0,0,0,0,0,hello @scandalofmoney there is indeed life aft...,2019
675177,0,0,1,0,0,0,0,0,0,0,0,0,online course deals! learn how to use google's...,2019


# Get 1k samples

In [10]:
# get 100k samples
df_1_list = [get_100k(n=1000, df=df_full, year=year) for year in years]

df_1 = pd.concat(df_1_list, ignore_index=True)

OUTPUT = parent_dir/'AI Project'/'Avery_output'/'AI_sequential_topic'
outname ='AI_topic_company_1k.csv'


df_1.to_csv(os.path.join(str(OUTPUT),outname), index = False)

In [ ]:
# Plotting tools
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import re
import numpy as np
import pandas as pd

#NLTK
import nltk
# nltk.download('stopwords')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, LdaModel

# spacy for lemmatization
import spacy
if 'nlp' not in locals():
    # nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])
    nlp = spacy.load('en', disable=['parser', 'ner'])

import gc
import os
from glob import glob
from IPython.display import Audio, display

# Text Analysis (uncomment if running for first time)
# ! wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
# ! unzip mallet-2.0.8.zip
MALLET_PATH = 'mallet-2.0.8/bin/mallet'